# Gradio & Hugging Face Pipelines Tutorial

This tutorial shows how to build Gradio apps using **Blocks**, **Row**, and **Column** layouts with common components like **images**, **text inputs**, **sliders**, **buttons**, and **live outputs**. Then, it demonstrates how to integrate Hugging Face `transformers` **pipelines** into Gradio to add pretrained NLP capability (sentiment classification) with only a few lines of code.

We build two small apps:

1. **Poster Critic Studio (creative app)**: Upload an image, pick a mood, adjust sliders, and get a movie-poster tagline + critic score.
2. **Hugging Face Sentiment Tool**: Use a pretrained sentiment pipeline inside Gradio to classify any text.


## Gradio Basics: Blocks, Rows, Columns, and Components

- **`gr.Blocks()`** lets you build custom multi-component apps (more flexible than `gr.Interface()`).
- **`gr.Row()`** places components side-by-side horizontally.
- **`gr.Column()`** stacks components vertically, and multiple columns can sit inside a row.
- Common components used in this tutorial:
  - `gr.Image()` for image upload/camera input
  - `gr.Textbox()` for text input/output
  - `gr.Slider()` for numeric controls
  - `gr.Button()` for triggering actions
  - `gr.Markdown()` and `gr.Label()` for formatted outputs
- Events:
  - **`.click()`** runs a Python function when a button is pressed
  - **`.change()`** runs a function when a component’s value changes


In [1]:
#Install required libraries
!pip -q install gradio transformers torch pillow

In [2]:
import gradio as gr
import numpy as np
from PIL import Image

#Hugging Face pipeline (pretrained model wrapper)
from transformers import pipeline

## App 1: Poster Critic Studio (Creative App)

**Idea:** Upload an image and choose a mood (e.g., mysterious, chaotic). The app generates:
- A poster-like tagline
- A critic score (0–100) influenced by sliders
- A simple palette guess using average RGB, brightness, and contrast


In [3]:
def image_style_features(img: Image.Image):
    """
    Extract very simple image "style" features using pixel statistics:
    - mean RGB (rough dominant color)
    - brightness (mean of all pixel values)
    - contrast (std of all pixel values)
    """
    arr = np.array(img.convert("RGB"), dtype=np.float32) / 255.0
    mean_rgb = arr.mean(axis=(0, 1))
    brightness = float(arr.mean())
    contrast = float(arr.std())
    return mean_rgb, brightness, contrast


def palette_label(mean_rgb):
    """
    Convert average RGB into a friendly palette label.
    """
    r, g, b = mean_rgb
    if r > g and r > b:
        return "warm / red-leaning"
    if b > r and b > g:
        return "cool / blue-leaning"
    if g > r and g > b:
        return "fresh / green-leaning"
    return "balanced / neutral"


def poster_critic_studio(img, mood, drama, weirdness, tagline_seed):
    """
    Inputs:
      img: PIL image (from Gradio)
      mood: dropdown text
      drama: slider (0-10)
      weirdness: slider (0-10)
      tagline_seed: optional text

    Outputs:
      tagline (str), critic score (int), palette note (str)
    """
    if img is None:
        return "Please upload an image first.", 0, "No image uploaded."

    mean_rgb, brightness, contrast = image_style_features(img)
    palette = palette_label(mean_rgb)

    #Small library of mood phrases
    mood_phrases = {
        "mysterious": ["A secret waits behind every shadow.", "Nothing is what it seems."],
        "romantic":   ["Love arrives without asking.", "Two hearts, one impossible choice."],
        "chaotic":    ["Everything breaks at once.", "The plan was simple. Then reality showed up."],
        "heroic":     ["Courage is a decision.", "One step forward. No way back."],
        "melancholy": ["Some goodbyes last forever.", "The quiet hurts the most."],
    }

    phrases = mood_phrases.get(mood.lower(), ["A story begins where certainty ends."])
    base = phrases[int((drama + weirdness) % len(phrases))]

    seed_part = f" Seed: {tagline_seed.strip()}" if tagline_seed and tagline_seed.strip() else ""
    tagline = f"{base}{seed_part}"

    #Simple score logic (0–100)
    score = 50
    score += int(drama * 3)                 # up to +30
    score += int(weirdness * 2)             # up to +20
    score += int(contrast * 40)             # adds visual punch
    score += int((brightness - 0.45) * 30)  # small brightness effect
    score = max(0, min(100, score))

    palette_note = f"Palette: **{palette}** | Brightness: {brightness:.2f} | Contrast: {contrast:.2f}"
    return tagline, score, palette_note

In [4]:
#Quick test: ensures the function works without launching Gradio
dummy = Image.fromarray((np.random.rand(256, 256, 3) * 255).astype(np.uint8))
print(poster_critic_studio(dummy, "chaotic", 8, 7, "finals week"))

('The plan was simple. Then reality showed up. Seed: finals week', 100, 'Palette: **warm / red-leaning** | Brightness: 0.50 | Contrast: 0.29')


## App 2: Hugging Face Transformers Pipelines in Gradio

Hugging Face `pipeline()` provides a very simple interface to load a pretrained model for a task.

In this tutorial we use:
- `pipeline("sentiment-analysis")`

This returns a label (POSITIVE/NEGATIVE) and a confidence score. We will:
1. Analyze the sentiment of any text (Sentiment Tool tab)
2. Automatically analyze the sentiment of the tagline generated in App 1


In [5]:
#Load a pretrained sentiment pipeline
sentiment_pipe = pipeline("sentiment-analysis")


def analyze_sentiment(text):
    """
    Takes text and returns a small dictionary that Gradio can display.
    """
    if text is None or not text.strip():
        return {"error": "Please enter text."}

    out = sentiment_pipe(text)[0]  # pipeline returns a list
    return {"label": out["label"], "confidence": float(out["score"])}

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [6]:
with gr.Blocks(title="Gradio + Hugging Face Tutorial") as demo:
    gr.Markdown("# Gradio + Hugging Face Pipelines Tutorial")
    gr.Markdown("Two tabs: (1) a creative image-based app and (2) a Hugging Face sentiment tool.")

    with gr.Tabs():
      #Creative App
        with gr.Tab("Poster Critic Studio"):
            gr.Markdown("## Poster Critic Studio (creative app)")
            gr.Markdown("Upload an image, pick a mood, adjust sliders, and generate a tagline + score.")

            with gr.Row():
                with gr.Column():
                    img_in = gr.Image(type="pil", label="Upload or take a photo", height=320)

                    mood = gr.Dropdown(
                        choices=["mysterious", "romantic", "chaotic", "heroic", "melancholy"],
                        value="mysterious",
                        label="Poster mood",
                    )

                    drama = gr.Slider(0, 10, value=6, step=1, label="Drama level")
                    weirdness = gr.Slider(0, 10, value=3, step=1, label="Weirdness level")

                    tagline_seed = gr.Textbox(
                        label="Optional tagline seed",
                        placeholder="e.g., midnight train, lost city, final exam week...",
                    )

                    with gr.Row():
                        generate_btn = gr.Button("Generate")
                        clear_btn = gr.Button("Clear")

                with gr.Column():
                    tagline_out = gr.Textbox(label="Generated Tagline")
                    score_out = gr.Number(label="Critic Score (0–100)")
                    palette_out = gr.Markdown()

                    gr.Markdown("### Tagline Sentiment (Hugging Face pipeline)")
                    tagline_sentiment = gr.Label(label="Sentiment")

            #Generate outputs
            generate_btn.click(
                fn=poster_critic_studio,
                inputs=[img_in, mood, drama, weirdness, tagline_seed],
                outputs=[tagline_out, score_out, palette_out],
            )

            #Auto-run sentiment on the generated tagline
            tagline_out.change(
                fn=analyze_sentiment,
                inputs=tagline_out,
                outputs=tagline_sentiment,
            )

            #Clear everything
            clear_btn.click(
                fn=lambda: (None, "mysterious", 6, 3, "", "", 0, "", None),
                inputs=[],
                outputs=[img_in, mood, drama, weirdness, tagline_seed, tagline_out, score_out, palette_out, tagline_sentiment],
            )
        #HF pipeline demo
        with gr.Tab("Sentiment Tool"):
            gr.Markdown("## Hugging Face Sentiment Tool")
            gr.Markdown("Type any text and the pretrained model will classify its sentiment.")

            with gr.Row():
                with gr.Column():
                    text_in = gr.Textbox(label="Text to analyze", placeholder="Paste a sentence here...")
                    analyze_btn = gr.Button("Analyze")

                with gr.Column():
                    sentiment_out = gr.Label(label="Sentiment output")

            analyze_btn.click(
                fn=analyze_sentiment,
                inputs=text_in,
                outputs=sentiment_out,
            )

demo.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ecfdd69caa747b52c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## How to Use

### Poster Critic Studio
1. Upload or take a photo.
2. Choose a mood and adjust sliders.
3. Click Generate to produce a tagline + score + palette note.
4. The tagline sentiment is automatically analyzed using a Hugging Face pipeline.

### Sentiment Tool
1. Type any sentence.
2. Click Analyze to get the sentiment label and confidence score.
